In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths
import os
import os.path as osp
import cv2
import logging
import argparse
import motmetrics as mm
import numpy as np
import torch

from tracker.multitracker import JDETracker
from tracking_utils import visualization as vis
from tracking_utils.log import logger
from tracking_utils.timer import Timer
from tracking_utils.evaluation import Evaluator
import datasets.dataset.jde as datasets

from tracking_utils.utils import mkdir_if_missing
from opts import opts



class VideoWriter:
    def __init__(self,save_path,dataloader):
        self.frame_rate = dataloader.frame_rate
        self.height = dataloader.vh
        self.width = dataloader.vw
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.writer_obj = cv2.VideoWriter(f'{save_path}/output.avi', fourcc, self.frame_rate, (self.width,self.height))

    def write(self,frame):
        frame = cv2.resize(frame,(self.width,self.height))
        self.writer_obj.write(frame)



def eval_seq(opt, dataloader, data_type, save_dir=None, show_image=True):
    frame_rate = dataloader.frame_rate
    if save_dir:
        mkdir_if_missing(save_dir)
    tracker = JDETracker(opt, frame_rate=frame_rate)
    timer = Timer()
    results = []
    frame_id = 0
    writer = VideoWriter(save_dir,dataloader)
    for path, img, img0 in dataloader:
        # if frame_id % 20 == 0:
        #     logger.info('Processing frame {} ({:.2f} fps)'.format(frame_id, 1. / max(1e-5, timer.average_time)))

        # run tracking
        timer.tic()
        blob = torch.from_numpy(img).cuda().unsqueeze(0)
        online_targets = tracker.update(blob, img0)
        online_tlwhs = []
        online_ids = []
        for t in online_targets:
            tlwh = t.tlwh
            tid = t.track_id
            vertical = tlwh[2] / tlwh[3] > 1.6
            if tlwh[2] * tlwh[3] > opt.min_box_area and not vertical:
                online_tlwhs.append(tlwh)
                online_ids.append(tid)
        timer.toc()
        # save results
        results.append((frame_id + 1, online_tlwhs, online_ids))
        if show_image or save_dir is not None:
            online_im = vis.plot_tracking(img0, online_tlwhs, online_ids, frame_id=frame_id,
                                          fps=1. / timer.average_time)
        if show_image:
            cv2.imshow('online_im', online_im)
        if save_dir is not None:
            # cv2.imwrite(os.path.join(save_dir, '{:05d}.jpg'.format(frame_id)), online_im)
            writer.write(online_im)
        frame_id += 1
    # save results
    #write_results(result_filename, results, data_type)
    print("***************************** DONE *****************************")

def main(opt, data_root='/data/MOT16/train', det_root=None, seqs=('MOT16-05',), exp_name='demo',
         save_images=False, save_videos=False, show_image=True):
    logger.setLevel(logging.INFO)
    result_root = os.path.join(data_root, '..', 'results', exp_name)
    #print(data_root)
    mkdir_if_missing(result_root)
    data_type = 'mot'

    # run tracking
    accs = []
    n_frame = 0
    timer_avgs, timer_calls = [], []
    for seq in seqs:
        output_dir = os.path.join(data_root, '..', 'outputs', exp_name, seq) if save_images or save_videos else None
        logger.info('start seq: {}'.format(seq))
        dataloader = datasets.LoadImages(osp.join(data_root, seq, 'img1'), opt.img_size)
        result_filename = os.path.join(result_root, '{}.txt'.format(seq))
        meta_info = open(os.path.join(data_root, seq, 'seqinfo.ini')).read()
        frame_rate = int(meta_info[meta_info.find('frameRate') + 10:meta_info.find('\nseqLength')])
        nf, ta, tc = eval_seq(opt, dataloader, data_type, result_filename,
                              save_dir=output_dir, show_image=show_image, frame_rate=frame_rate)
        n_frame += nf
        timer_avgs.append(ta)
        timer_calls.append(tc)

        # eval
        logger.info('Evaluate seq: {}'.format(seq))
        evaluator = Evaluator(data_root, seq, data_type)
        accs.append(evaluator.eval_file(result_filename))
        if save_videos:
            output_video_path = osp.join(output_dir, '{}.mp4'.format(seq))
            cmd_str = 'ffmpeg -f image2 -i {}/%05d.jpg -c:v copy {}'.format(output_dir, output_video_path)
            os.system(cmd_str)
    timer_avgs = np.asarray(timer_avgs)
    timer_calls = np.asarray(timer_calls)
    all_time = np.dot(timer_avgs, timer_calls)
    avg_time = all_time / np.sum(timer_calls)
    logger.info('Time elapsed: {:.2f} seconds, FPS: {:.2f}'.format(all_time, 1.0 / avg_time))

    # get summary
    metrics = mm.metrics.motchallenge_metrics
    mh = mm.metrics.create()
    summary = Evaluator.get_summary(accs, seqs, metrics)
    strsummary = mm.io.render_summary(
        summary,
        formatters=mh.formatters,
        namemap=mm.io.motchallenge_metric_names
    )
    print(strsummary)
    Evaluator.save_summary(summary, os.path.join(result_root, 'summary_{}.xlsx'.format(exp_name)))



In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os


class opts(object):
    def __init__(self):
        # basic experiment setting
        self.task = 'mot'
        self.dataset = 'jde'
        self.exp_id = 'default'
        self.test = True
        #self.parser.add_argument('--load_model', default='../models/ctdet_coco_dla_2x.pth',
                                 #help='path to pretrained model')
        self.load_model = ''
        self.resume = True

        # system
        self.gpus = '0, 1'
        self.num_workers = 8
        self.not_cuda_benchmark = True
        self.seed = 317
        # log
        self.print_iter = 0
        self.hide_data_time = True
        self.save_all = True
        self.metric = 'loss'
        self.vis_thresh = 0.5
        # model
        self.arch = 'dla_34'
        self.head_conv = -1
        self.down_ratio = 4
        # input
        self.input_res = -1
        self.input_h = -1
        self.input_w = -1
        # train
        self.lr = 1e-4
        self.lr_step = '20,27'
        self.num_epochs = 30
        self.batch_size = 12
        self.master_batch_size = -1
        self.num_iters = -1
        self.val_intervals = 5
        self.trainval = True
        # test
        self.K = 128
        self.not_prefetch_test = True
        self.fix_res = True
        self.keep_res = True
        # tracking
        self.test_mot16 = False
        self.val_mot15 = False
        self.test_mot15 = False
        self.val_mot16 = False
        self.test_mot17 = False
        self.val_mot17 = False
        self.val_mot20 = False
        self.test_mot20 = False
        self.conf_thres = 0.6
        self.det_thres = 0.3
        self.nms_thres = 0.4
        self.track_buffer = 30
        self.min_box_area = 200
        self.input_video = '../videos/MOT16-03.mp4'
        self.output_format = 'video'
        self.output_root = '../results'

        # mot
        self.data_cfg = '../src/lib/cfg/data.json'
        self.data_dir = '../../data/yfzhang/MOT/JDE'

        # loss
        self.mse_loss = True

        self.reg_loss = 'l1'
        self.hm_weight = 1
        self.off_weight = 1
        self.wh_weight = 0.1
        self.id_loss = 'ce'
        self.id_weight = 1.0
        self.reid_dim = 512

        self.norm_wh = True
        self.dense_wh = True
        self.cat_spec_wh = True
        self.not_reg_offset = False

    def parse(self, args=''):
        opt = self
        opt.gpus_str = opt.gpus
        opt.gpus = [int(gpu) for gpu in opt.gpus.split(',')]
        opt.gpus = [i for i in range(len(opt.gpus))] if opt.gpus[0] >=0 else [-1]
        opt.lr_step = [int(i) for i in opt.lr_step.split(',')]

        opt.fix_res = not opt.keep_res
        print('Fix size testing.' if opt.fix_res else 'Keep resolution testing.')
        opt.reg_offset = not opt.not_reg_offset

        if opt.head_conv == -1: # init default head_conv
          opt.head_conv = 256 if 'dla' in opt.arch else 256
        opt.pad = 31
        opt.num_stacks = 1

        if opt.trainval:
          opt.val_intervals = 100000000

        if opt.master_batch_size == -1:
          opt.master_batch_size = opt.batch_size // len(opt.gpus)
        rest_batch_size = (opt.batch_size - opt.master_batch_size)
        opt.chunk_sizes = [opt.master_batch_size]
        for i in range(len(opt.gpus) - 1):
          slave_chunk_size = rest_batch_size // (len(opt.gpus) - 1)
          if i < rest_batch_size % (len(opt.gpus) - 1):
            slave_chunk_size += 1
          opt.chunk_sizes.append(slave_chunk_size)
        print('training chunk_sizes:', opt.chunk_sizes)

        opt.root_dir = os.path.join(os.path.dirname('__file__'), '..', '..')
        opt.exp_dir = os.path.join(opt.root_dir, 'exp', opt.task)
        opt.save_dir = os.path.join(opt.exp_dir, opt.exp_id)
        opt.debug_dir = os.path.join(opt.save_dir, 'debug')
        print('The output will be saved to ', opt.save_dir)

        if opt.resume and opt.load_model == '':
          model_path = opt.save_dir[:-4] if opt.save_dir.endswith('TEST') \
                      else opt.save_dir
          opt.load_model = os.path.join(model_path, 'model_last.pth')
        return opt

    def update_dataset_info_and_set_heads(self, opt, dataset):
        input_h, input_w = dataset.default_resolution
        opt.mean, opt.std = dataset.mean, dataset.std
        opt.num_classes = dataset.num_classes

        # input_h(w): opt.input_h overrides opt.input_res overrides dataset default
        input_h = opt.input_res if opt.input_res > 0 else input_h
        input_w = opt.input_res if opt.input_res > 0 else input_w
        opt.input_h = opt.input_h if opt.input_h > 0 else input_h
        opt.input_w = opt.input_w if opt.input_w > 0 else input_w
        opt.output_h = opt.input_h // opt.down_ratio
        opt.output_w = opt.input_w // opt.down_ratio
        opt.input_res = max(opt.input_h, opt.input_w)
        opt.output_res = max(opt.output_h, opt.output_w)

        if opt.task == 'mot':
          opt.heads = {'hm': opt.num_classes,
                       'wh': 2 if not opt.cat_spec_wh else 2 * opt.num_classes,
                       'id': opt.reid_dim}
          if opt.reg_offset:
            opt.heads.update({'reg': 2})
          opt.nID = dataset.nID
          opt.img_size = (1088, 608)
        else:
          assert 0, 'task not defined!'
        print('heads', opt.heads)
        return opt

    def init(self, args=''):
        default_dataset_info = {
          'mot': {'default_resolution': [608, 1088], 'num_classes': 1,
                    'mean': [0.408, 0.447, 0.470], 'std': [0.289, 0.274, 0.278],
                    'dataset': 'jde', 'nID': 14455},
        }
        class Struct:
          def __init__(self, entries):
            for k, v in entries.items():
              self.__setattr__(k, v)
        opt = self.parse(args)
        dataset = Struct(default_dataset_info[opt.task])
        opt.dataset = dataset.dataset
        opt = self.update_dataset_info_and_set_heads(opt, dataset)
        return opt


In [3]:
video_path = #path of the video to load

In [4]:
dl = datasets.LoadVideo(video_path, (1088,608))  #load the dataset

Lenth of the video: 4706 frames


In [ ]:
opt = opts().init()
opt.load_model = # path of the saved model
show_image = False
output_dir = #path to save the video
eval_seq(opt, dl, 'mot',save_dir=output_dir, show_image=show_image)